In [1]:
from pyspark.sql import SparkSession

In [2]:
spark =SparkSession.builder.appName('Logistic Regression').getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
data =spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [5]:
data.take(1)[0].asDict()

{'Names': 'Cameron Williams',
 'Age': 42.0,
 'Total_Purchase': 11066.8,
 'Account_Manager': 0,
 'Years': 7.22,
 'Num_Sites': 8.0,
 'Onboard_date': datetime.datetime(2013, 8, 30, 7, 0, 40),
 'Location': '10265 Elizabeth Mission Barkerburgh, AK 89518',
 'Company': 'Harvey LLC',
 'Churn': 1}

In [6]:
data.printSchema()


root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [7]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [8]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [9]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase','Years','Num_Sites'],outputCol='features')

In [11]:
output = assembler.transform(data)

In [12]:
output.take(1)[0].asDict()

{'Names': 'Cameron Williams',
 'Age': 42.0,
 'Total_Purchase': 11066.8,
 'Account_Manager': 0,
 'Years': 7.22,
 'Num_Sites': 8.0,
 'Onboard_date': datetime.datetime(2013, 8, 30, 7, 0, 40),
 'Location': '10265 Elizabeth Mission Barkerburgh, AK 89518',
 'Company': 'Harvey LLC',
 'Churn': 1,
 'features': DenseVector([42.0, 11066.8, 7.22, 8.0])}

In [13]:
final_data = output.select(['Churn','features'])

In [14]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [15]:
classifier = LogisticRegression(featuresCol='features',labelCol='Churn',predictionCol='predicted churn status')

In [16]:
model = classifier.fit(train_data)

In [17]:
train_eval = model.summary

In [18]:
train_eval.predictions.summary().show()

+-------+-------------------+----------------------+
|summary|              Churn|predicted churn status|
+-------+-------------------+----------------------+
|  count|                626|                   626|
|   mean|0.17092651757188498|   0.13099041533546327|
| stddev| 0.3767457766076394|   0.33765967705366784|
|    min|                0.0|                   0.0|
|    25%|                0.0|                   0.0|
|    50%|                0.0|                   0.0|
|    75%|                0.0|                   0.0|
|    max|                1.0|                   1.0|
+-------+-------------------+----------------------+



In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [20]:
evaluation = model.evaluate(test_data)

In [21]:
evaluation.predictions.show()

+-----+--------------------+--------------------+--------------------+----------------------+
|Churn|            features|       rawPrediction|         probability|predicted churn status|
+-----+--------------------+--------------------+--------------------+----------------------+
|    0|[22.0,11254.38,4....|[4.71724099616739...|[0.99113940253056...|                   0.0|
|    0|[28.0,11204.23,3....|[1.32593497056015...|[0.79016743627522...|                   0.0|
|    0|[28.0,11245.38,6....|[3.47310332836606...|[0.96991271216095...|                   0.0|
|    0|[29.0,9378.24,4.9...|[4.49546455554200...|[0.98896366472517...|                   0.0|
|    0|[29.0,9617.59,5.4...|[4.18554589025064...|[0.98501409547543...|                   0.0|
|    0|[29.0,11274.46,4....|[4.60613093664496...|[0.99010842366840...|                   0.0|
|    0|[29.0,13240.01,4....|[6.86398834743350...|[0.99895635150925...|                   0.0|
|    0|[30.0,8403.78,4.1...|[6.17823705548318...|[0.99793021

In [22]:
final_eval = BinaryClassificationEvaluator(labelCol='Churn',rawPredictionCol='predicted churn status')

In [23]:
a=final_eval.evaluate(evaluation.predictions)

In [24]:
final_eval.getRawPredictionCol()

'predicted churn status'

In [25]:
evaluation.areaUnderROC

0.9093425953891067

In [26]:
evaluation.precisionByLabel

[0.9057377049180327, 0.6666666666666666]

In [27]:
final_lr_model = classifier.fit(final_data)

In [28]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [29]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [30]:
 test_new_customers = assembler.transform(new_customers)

In [31]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [32]:
final_results = final_lr_model.transform(test_new_customers)

In [34]:
final_results.select('Company','predicted churn status').show()

+----------------+----------------------+
|         Company|predicted churn status|
+----------------+----------------------+
|        King Ltd|                   0.0|
|   Cannon-Benson|                   1.0|
|Barron-Robertson|                   1.0|
|   Sexton-Golden|                   1.0|
|        Wood LLC|                   0.0|
|   Parks-Robbins|                   1.0|
+----------------+----------------------+

